In [ ]:
# IMPORTS
import sys
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
import torch.optim as optim
import matplotlib
import matplotlib.pyplot as plt
from timeit import default_timer as timer

In [ ]:
# Do the imports from other files in the project
from Model.DepthLSTM import DepthLSTM
from Train.hyperparameters import *
from Train.train_epoch_J import train_epoch_J
from Train.test_epoch_J import test_epoch_J

### Load Data

In [ ]:
path_videos = './Data/Preprocessed_J/'
list_videos = os.listdir(path_videos)
videos_data = []

In [ ]:
for video in list_videos:
    path_video = path_videos + video
    videos_data.append(np.load(path_video))

In [ ]:
videos_data[15].shape

### Define test set and train set

In [ ]:
testset_idx = [0, 4, 9, 12, 21]
trainset_idx = list(set(range(24)) - set(testset_idx))

In [ ]:
train_videos = [videos_data[i] for i in trainset_idx]
test_videos = [videos_data[i] for i in testset_idx]

### Construct train set and test set

In [ ]:
def joinVideos(videos_data):
    span_videos = []
    for video in videos_data:
        span_vid = np.ones(len(video))
        span_vid[0] = 0
        span_videos.append(span_vid)
    span_videos = np.concatenate(span_videos)
    videos_data = np.concatenate(videos_data)
    return videos_data, span_videos


def reshapeBatches(vid_data, BATCH_SIZE, SEQ_LEN):
    num_batches = vid_data.shape[0] // (BATCH_SIZE*SEQ_LEN)
    cut_vid = vid_data[:num_batches*BATCH_SIZE*SEQ_LEN,]
    reshaped_vid = cut_vid.reshape(BATCH_SIZE, num_batches, SEQ_LEN, -1)
    reordered_vid = np.transpose(reshaped_vid, (1, 2, 0, 3))
    
    return reordered_vid


def splitInputOutput(vid_data):
    ground_idx = list(range(2,201,3))
    input_idx = list(set(range(201)) - set(ground_idx))
    
    input_vid = vid_data[:,:,:,input_idx]
    output_vid = vid_data[:,:,:,ground_idx]
    
    return input_vid, output_vid

In [ ]:
def create_stateful_dataset(videos, shuffle = True):
    if shuffle:
        random.shuffle(videos)
        
    videos_set, span_videos = joinVideos(videos)
    
    dataset = reshapeBatches(videos_set, BATCH_SIZE, SEQ_LEN)
    span_videos = reshapeBatches(span_videos, BATCH_SIZE, SEQ_LEN)
    
    dataset, grounddataset = splitInputOutput(dataset)
    
    dataset = dataset.astype(np.float32)
    grounddataset = grounddataset.astype(np.float32)
    
    return dataset, grounddataset, span_videos

### Define evaluation metrics

In [ ]:
def computeThresholds(input_data, gt, reference):
    new_shape = input_data.shape[:-1]
    new_shape = list(new_shape) + [1]
    thresholds = np.zeros(new_shape)
    for i in range(input_data.shape[0]):
        for j in range(input_data.shape[1]):
            for k in range(input_data.shape[2]):
                a_x = input_data[i,j,k,reference[0]*2]
                a_y = input_data[i,j,k,reference[0]*2 + 1]
                a_z = gt[i,j,k,reference[0]]
                b_x = input_data[i,j,k,reference[1]*2]
                b_y = input_data[i,j,k,reference[1]*2 + 1]
                b_z = gt[i,j,k,reference[1]]
                
                a = np.array([a_x-b_x, a_y-b_y, a_z-b_z])
                dist = np.linalg.norm(a)
                
                thresholds[i,j,k,:] = dist
                
    return thresholds

In [ ]:
def evaluate_model(model, input_data, gt, span, reference, alpha = 0.1, hands = False):
    loss_function = nn.L1Loss()
    test_epoch_loss, predY = test_epoch_J(model, input_data, gt, span, loss_function)
    predY = np.squeeze(np.array(predY))
    
    thresholds = computeThresholds(input_data, gt, reference)
    classification = abs(predY - gt) <= thresholds*alpha
    
    classification_hands = classification[:,:,:,25:]
    
    #print(classification_hands.shape)
    
    # print(classification.sum(), classification.sum()/classification.size)
    
    #computePCK(input_data, gt, output, , alpha)
    
    if hands:
        return classification_hands.sum()/classification_hands.size
    else:
        return classification.sum()/classification.size

In [ ]:
trainset, groundtrainset, span_videos_train = create_stateful_dataset(train_videos, shuffle = False)
testset, groundtestset, span_videos_test = create_stateful_dataset(test_videos, shuffle = False)

In [ ]:
# Take into account only the hands
hands_flag = False

### Load model 1

In [ ]:
path_model = "./Output/J_300.pt"
model = torch.load(path_model)

In [ ]:
model

In [ ]:
# Reference is the two joints of the bone that we take as reference.
reference = (0, 2)

t = evaluate_model(model, testset, groundtestset, span_videos_test, reference, hands = hands_flag)
print(t)

### Evaluate with different alphas

In [ ]:
alpha_v = np.arange(0.01, 0.5, 0.01)

In [ ]:
acc_clean = []
for alpha in alpha_v:
    acc_clean.append(evaluate_model(model, testset, groundtestset, span_videos_test, reference, alpha=alpha, hands = hands_flag))

### Load model 2

In [ ]:
path_model_2 = "./Output/M_300.pt"
model_2 = torch.load(path_model_2)

model_2

In [ ]:
t = evaluate_model(model_2, testset, groundtestset, span_videos_test, reference, hands = hands_flag)
print(t)

In [ ]:
acc_raw = []
for alpha in alpha_v:
    acc_raw.append(evaluate_model(model_2, testset, groundtestset, span_videos_test, reference, alpha=alpha, hands = hands_flag))

In [ ]:
plt.plot(alpha_v, acc_clean, label = 'preprocessed')
plt.plot(alpha_v, acc_raw, label = 'raw')
plt.xlabel('alpha')
plt.ylabel('PCK')
plt.legend()

### Losses

In [ ]:
def read_loss_file(path):
    
    with open("./Output/" + path, 'r') as f:
        loss = f.readlines()
        loss = [float(l) for l in loss]
        
    return loss

def get_losses(name):
    train_loss_path = name + "_train_loss.txt"
    test_loss_path = name + "_test_loss.txt"
    
    train_loss = read_loss_file(train_loss_path)
    test_loss = read_loss_file(test_loss_path)
    
    return train_loss, test_loss

In [ ]:
name_m = "M"

train_loss_m, test_loss_m = get_losses(name_m)

In [ ]:
plt.plot(train_loss_m, label = 'train')
plt.plot(test_loss_m, label = 'test')
plt.legend()

In [ ]:
name_j = "J"

train_loss_j, test_loss_j = get_losses(name_j)

In [ ]:
plt.plot(train_loss_j, label = 'train')
plt.plot(test_loss_j, label = 'test')